In [1]:
# Erasmus+ ICCT project (2018-1-SI01-KA203-047081)

# Toggle cell visibility

from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Promijeni vidljivost <a href="javascript:code_toggle()">ovdje</a>.''')
display(tag)

# Hide the code completely

# from IPython.display import HTML
# tag = HTML('''<style>
# div.input {
#     display:none;
# }
# </style>''')
# display(tag)


## Dinamika bočnog položaja lunarnog prizemljivača

Ovaj primjer pokazuje bočnu dinamiku lunarnog prizemljivača pomoću diferencijalnih jednadžbi. Te se jednadžbe zatim preformuliraju u oblik prostora stanja.

<img src="Images\lunarlander_hr.png" alt="drawing" width="500x800">

Sustav lunarnog prizemljivača predstavljen je na gornjoj slici; njegov vertikalni spust usporen je pomoću vertikalnog potisnika koji proizvodi konstantnu silu $F$. Horizontalno kretanje može se postići laganim naginjanjem prizemljivača za kut $\theta$, naginjanjem se stvara bočna sila približno jednaka $F\theta$. Nagib se postiže generiranjem momenta $T$ pomoću skupa upravljačkih raketa (maksimalni moment = 500 Nm). Pretpostavlja se da je atmosferski otpor zanemariv, a vrijednosti parametara navedene su u donjoj tablici.

| Parametar |                     Vrijednost |
|-----------|-------------------------------:|
|$m$        |                        1000 kg |
|$J$        |            1000 kg$\text{m}^2$ |
|$F$        |                         1500 N |

### Diferencijalna jednadžba
Korištenjem Newton-Eulerovih jedadžbi, moguće je pisati:

\begin{cases}
J\ddot{\theta}=T \\
m\ddot{z}=F\theta,
\end{cases}

gdje je $z$ lateralna pozicija (bočni položaj).

### Prostor stanja
Budući da su dvije diferencijalne jednadžbe drugog reda, broj potrebnih stanja (kako bi se u potpunosti opisao razvoj sustava) je 4 (2 i 2). Definiranjem vektora stanja kao $\textbf{x}=[x_1,x_2,x_3,x_4]^T=[z,\dot{z},\theta,\dot{\theta}]^T$ i ulaza kao $u=T$, u prostoru stanja jednadžbe postaju:

$$
\dot{\textbf{x}}=\underbrace{\begin{bmatrix}0&1&0&0 \\ 0&0&F/m&0 \\ 0&0&0&1 \\ 0&0&0&0\end{bmatrix}}_{A}\textbf{x}+\underbrace{\begin{bmatrix}0\\0\\0\\1/J\end{bmatrix}}_{B}u
$$

### Kako koristiti ovaj interaktivni primjer?

Poigrajte se s parametrima sustava i pokušajte odgovoriti:
- Je li moguće postići konstantnu vrijednost $\theta$ primjenom impulsnog ili koračnog (step) ulaza? Zašto?
- Jesu li velike vrijednosti $\theta$ fizički zanimljive? Zašto?

In [2]:
#Preparatory Cell 

%matplotlib notebook
import control
import numpy
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt

%matplotlib inline
#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

In [3]:
#define matrixes
C = numpy.matrix([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
D = numpy.matrix([[0],[0],[0],[0]])
X0 = matrixWidget(4,1)


m = widgets.FloatSlider(
    value=1000,
    min=400,
    max=2000,
    step=1,
    description='$m$ [kg]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
F = widgets.FloatSlider(
    value=1500,
    min=10,
    max=5000,
    step=10,
    description='$F$ [N]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
J = widgets.FloatSlider(
    value=1000,
    min=500,
    max=2000,
    step=1,
    description=r'$J$ [kg$\text{m}^2$]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
u = widgets.FloatSlider(
    value=100,
    min=0,
    max=500,
    step=0.01,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
omega = widgets.FloatSlider(
    value=5,
    min=0,
    max=10.0,
    step=0.1,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [4]:
#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Click!',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Klikni za promjenu inicijalnih uvjeta',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

#define type of ipout 
SELECT = widgets.Dropdown(
    options=['impuls', 'step', 'sinus'],
    value='impuls',
    description='',
    disabled=False
)

In [5]:
def main_callback(X0, m, F, J, u, selu, omega, DW):
    a = numpy.matrix([[0,1,0,0],[0,0,F/m,0],[0,0,0,1],[0,0,0,0]])
    b = numpy.matrix([[0],[0],[0],[1/J]])

    eig = numpy.linalg.eig(a)
    sys = sss(a,b,C,D)
    
    if min(numpy.real(abs(eig[0]))) != 0:
        T = numpy.linspace(0,10/min(numpy.real(abs(eig[0]))),1000)
    else:
        if max(numpy.real(abs(eig[0]))) != 0:
            T = numpy.linspace(0,10/max(numpy.real(abs(eig[0]))),1000)
        else:
            T = numpy.linspace(0,30,1000)
        
    if selu == 'impuls': #selu
        U = [0 for t in range(0,len(T))]
        U[0] = u
        y = control.forced_response(sys,T,U,X0)
    if selu == 'step':
        U = [u for t in range(0,len(T))]
        y = control.forced_response(sys,T,U,X0)
    if selu == 'sinus':
        U = u*numpy.sin(omega*T)
        y = control.forced_response(sys,T,U,X0)
    
    fig=plt.figure(num=1,figsize=[15, 8])
    fig.add_subplot(221)
    plt.plot(T,y[1][0])
    plt.grid()
    plt.xlabel('time [s]')
    plt.ylabel(r'$z$ [m]')
    
    fig.add_subplot(222)
    plt.plot(T,y[1][1])
    plt.grid()
    plt.xlabel('time [s]')
    plt.ylabel(r'$\dot{z}$ [m/s]')
    
    fig.add_subplot(223)
    plt.plot(T,y[1][2])
    plt.grid()
    plt.xlabel('time [s]')
    plt.ylabel(r'$\theta$ [rad]')
    
    fig.add_subplot(224)
    plt.plot(T,y[1][3])
    plt.grid()
    plt.xlabel('time [s]')
    plt.ylabel(r'$\dot{\theta}$ [rad/s]')
    
    #display(Markdown('The A matrix is: $%s$ and the eigenvalues are: $%s$' % (bmatrix(a),eig[0])))

#create a graphic structure to hold all widgets 
alltogether =  widgets.VBox([widgets.HBox([widgets.VBox([m,
                                           J,
                                           F]),
                            widgets.HBox([widgets.VBox([widgets.Label('Odaberite tip ulaza:',border=3),
                                                        widgets.Label('$u$:',border=3),
                                                        widgets.Label('omega [rad/s]:',border=3)]),
                                          widgets.VBox([SELECT,u,omega])])]),
                             widgets.HBox([widgets.Label('Inicijalno stanje X0:',border=3),X0,
                                           widgets.Label('Klikni za promjenu inicijalnih uvjeta:',border=3),START])])

out = widgets.interactive_output(main_callback,{'X0':X0, 'm': m, 'F': F, 'J': J, 'u': u, 'selu': SELECT, 'omega':omega, 'DW':DW})
#out.layout.height = '300px'
display(out,alltogether)

Output()

In [6]:
#create dummy widget 2
DW2 = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))
DW2.value = -1

#create button widget
START2 = widgets.Button(
    description='Prikaži odgovore',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Klikni za pregled odgovora',
    icon='check'
)
                       
def on_start_button_clicked2(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW2.value> 0 :
        DW2.value = -1
    else: 
        DW2.value = 1
    pass
START2.on_click(on_start_button_clicked2)

def main_callback2(DW2):
    if DW2 > 0:
        display(Markdown('''> Odgovori:
                            >* Ne, nije moguće. Dinamika varijable $\\theta$ je dinamika takozvanog dvostrukog integratora: vrijednost $\\theta$ može se dobiti integriranjem dvostrukog ulaza (prikladno skaliranom inverzijom inercije $J$). U smislu predstavljanja prostorom stanja, to naglašava činjenica da matrica dinamike ima dvije svojstvene vrijednosti u 0 koje tvore Jordanov blok dimenzije 2. Dakle, modovi sustava su $1$ i $t$, a ovaj potonji je različit.
                            >* Ako vrijednosti $\\theta$ postanu velike, aproksimacija da je bočna sila približno jednaka $F\\theta$ više nije valjana, pa ovaj model gubi primjenjivost. '''))
    else:
        display(Markdown(''))

#create a graphic structure to hold all widgets 
alltogether2 =  widgets.VBox([START2])

out2 = widgets.interactive_output(main_callback2,{'DW2':DW2})
#out.layout.height = '300px'
display(out2,alltogether2)

Output()